In [1]:
import numpy as np
import pandas as pd
import jieba

In [2]:
def cut(string): return ' '.join(jieba.cut(string))

In [3]:
news_file = 'sqlResult_1558435.csv'

In [4]:
news_content = pd.read_csv(news_file, encoding='gb18030')

In [5]:
news_content.head(5)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
pure_content = pd.DataFrame()

In [7]:
pure_content['content'] = news_content['content']

In [8]:
pure_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 1 columns):
content    87054 non-null object
dtypes: object(1)
memory usage: 700.2+ KB


In [9]:
pure_content = pure_content.fillna('')

In [10]:
pure_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 1 columns):
content    89611 non-null object
dtypes: object(1)
memory usage: 700.2+ KB


In [11]:
pure_content['tokenized_content'] = pure_content['content'].apply(cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/_8/f14fxxnn7w13cd0l5x9hl4bm0000gn/T/jieba.cache
Loading model cost 0.955 seconds.
Prefix dict has been built succesfully.


In [12]:
pure_content.head(5)

,content,tokenized_content
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,此外 ， 自 本周 （ 6 月 12 日 ） 起 ， 除 小米 手机 6 等 15 款 机型...
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,骁龙 835 作为 唯一 通过 Windows 10 桌面 平台 认证 的 ARM 处理...
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,此前 的 一加 3T 搭载 的 是 3400mAh 电池 ， DashCharge 快充 规...
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,这是 6 月 18 日 在 葡萄牙 中部 大 佩德罗 冈 地区 拍摄 的 被 森林 大火 烧...
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,（ 原 标题 ： 44 岁 女子 跑 深圳 约会 网友 被 拒 ， 暴雨 中 裸身 奔走 …...


### Train the word vector

In [13]:
from gensim.models import FastText

In [14]:
# 拼接所有切词后的内容
with open('all_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(pure_content['tokenized_content'].tolist()))

In [16]:
from gensim.models.word2vec import LineSentence

In [17]:
model = FastText(LineSentence('all_corpus.txt'), window=5, size=35, iter=10, min_count=1)

In [18]:
model.wv['葡萄牙']

array([-1.4963443 , -1.2078925 , -2.5167086 ,  0.5643393 ,  0.6551536 ,
        1.1362991 ,  3.3342147 , -1.8987615 , -3.9575288 , -1.977357  ,
        1.9952084 , -0.7902811 , -3.4335809 ,  0.7119318 ,  0.18408641,
        0.34074834, -1.6191152 , -0.9183833 ,  2.924654  , -1.3066695 ,
        0.12851925,  4.3276215 ,  3.1752946 , -1.5045592 , -2.8394198 ,
       -1.0600727 , -0.45361567,  0.23963368, -1.5004656 , -2.0569327 ,
       -0.2110024 ,  3.5523028 ,  0.828277  ,  1.7665414 ,  0.8937745 ],
      dtype=float32)

In [19]:
tokeners = [t for l in pure_content['tokenized_content'].tolist() for t in l.split()]

In [20]:
tokeners[:10]

['此外', '，', '自', '本周', '（', '6', '月', '12', '日', '）']

### Counts the words occureence

In [21]:
from collections import Counter

In [22]:
tokener_counter = Counter(tokeners)

In [23]:
frequence = {w: count / len(tokeners) for w, count in tokener_counter.items()}

In [23]:
def SIF_sentence_embedding(sentence, alpha=1e-4):

    global frequence
    
    max_fre = max(frequence.values())
    
    words = cut(sentence).split()
    
    sentence_vec = np.zeros_like(model.wv['测试'])
    
    words = [w for w in words if w in model]
    
    for w in words:
        weight = alpha / (alpha + frequence.get(w, max_fre))
        sentence_vec += weight*model.wv[w]
        
    sentence_vec /= len(words)

    return sentence_vec

In [24]:
from scipy.spatial.distance import cosine

In [25]:
import re

In [26]:
def split_sentences(text, p='[。，,.]', filter_p='\s+'):
    f_p = re.compile(filter_p)
    text = re.sub(f_p, '', text)
    pattern = re.compile(p)
    split = re.split(pattern, text)
    return split

In [29]:
def softmax(logits):
    logits = logits - np.max(logits)
    with_exp = np.exp(logits)
    return with_exp / np.sum(with_exp)

In [64]:
def get_corr(text, embed_fn=SIF_sentence_embedding):
    if isinstance(text, list): text = ' '.join(text)
    
    sub_sentences = split_sentences(text)
    sen_vec = embed_fn(text)
    
    corr_score = {}
    
    for sen in sub_sentences:
        sub_sen_vec = embed_fn(sen)
        corr_score[sen] = cosine(sen_vec, sub_sen_vec)
    
    # 分句分数
    score_softmax = softmax([value for value in corr_score.values()])
    
    result_score = {}
    
    for index, key in enumerate(corr_score.keys()):
        result_score[key] = score_softmax[index]
        
    return sorted(corr_score.items(), key=lambda x:x[1], reverse=True)

In [65]:
def get_summarization(text, score_fn, sum_len):
    
    # 分句列表
    sub_sentences = split_sentences(text)
    
    # 分句分数
    ranking_sentences = score_fn(text)
    selected_sen = set()
    current_sen = ''
    
    for sen, _ in ranking_sentences:
        if len(current_sen)<sum_len:
            current_sen += sen
            selected_sen.add(sen)
        else:
            break
            
    summarized = []
    
    for sen in sub_sentences:
        if sen in selected_sen:
            summarized.append(sen)
            
    return summarized

In [66]:
def get_summarization_by_sen_emb(text,max_len=200):
    return get_summarization(text,get_corr,max_len)

In [67]:
print(''.join(get_summarization_by_sen_emb(pure_content['content'].iloc[6])))

并于2013年退出中国市场但是今年3月份官方突然宣布回归中国那么从其中相关信息了解到八个Kryo260核心设计集成Adreno512GPU和X12LTE调制解调器但在多核新能上比去年骁龙820强单核改进也很明显不过由于OPPO与高通签署了排他性协议可以独占两三个月时间说明只要等独占时期一过按照之前被曝光的渲染图了解在5月份的媒体沟通会上惠普罗忠生表示：“我敢打赌12个月之后在座的各位手机都会换掉因为全面屏时代的到来”


/anaconda3/envs/Kstone/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [68]:
pure_content['content'].iloc[6]

'虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。那么，第一款夏普手机什么时候登陆中国呢？又会是怎么样的手机呢？\r\n近日，一款型号为 FS8016 的夏普神秘新机悄然出现在 GeekBench 的跑分库上。从其中相关信息了解到，这款机子并非旗舰定位，所搭载的是高通骁龙 660 处理器，配备有 4GB 的内存。骁龙 660 是高通今年最受瞩目的芯片之一，采用 14 纳米工艺，八个 Kryo 260 核心设计，集成 Adreno 512 GPU 和 X12 LTE 调制解调器。\r\n当前市面上只有一款机子采用了骁龙 660 处理器，那就是已经上市销售的 OPPO R11。骁龙 660 尽管并非旗舰芯片，但在多核新能上比去年骁龙 820 强，单核改进也很明显，所以放在今年仍可以让很多手机变成高端机。不过，由于 OPPO 与高通签署了排他性协议，可以独占两三个月时间。\r\n考虑到夏普既然开始测试新机了，说明只要等独占时期一过，夏普就能发布骁龙 660 新品了。按照之前被曝光的渲染图了解，夏普的新机核心竞争优势还是全面屏，因为从 2013 年推出全球首款全面屏手机 EDGEST 302SH 至今，夏普手机推出了多达 28 款的全面屏手机。\r\n在 5 月份的媒体沟通会上，惠普罗忠生表示：“我敢打赌，12 个月之后，在座的各位手机都会换掉。因为全面屏时代的到来，我们怀揣的手机都将成为传统手机。”\r\n'